In [ ]:
import os
import vesicle_analysis.io_utils as io
import vesicle_analysis.segment as seg
import vesicle_analysis.utils as uts
import vesicle_analysis.visualise as vis

# Define variables

In [ ]:
# Path to use for single image analysis
file_path = "../../../../20241205_PD21_65_notworking_crops/20210602_PD21_065_Vehicle_CCL19_z180_005_crop01.tif"
dir_path = "../../../../20241205_PD21_65_notworking_crops"

# Channel identifiers. E.g. if vesicles are the first channel use 0
dapi_ch = 0
vesicle_ch = 3

# Additional information for saving-file name. Can be empty, i.e. ''
channel_name = "channel3-rab"

# Use None for automatic threshold choosing. Override auto-threshold choosing
# by using either 'otsu' or 'yen'
method_vesicle = None

## Analyse individual image

In [ ]:
# Load image
img, pixel_size = io.read_image(file_path)
img_dapi = io.get_channel(img, dapi_ch)
img_ves = io.get_channel(img, vesicle_ch)
# Segment
nuc_mask, nuc_num = seg.segment_nucleus(img_dapi, pixel_size)
nuc_volume, nuc_volume_cal = uts.get_nucleus_volume(nuc_mask, pixel_size)
ves_mask, ves_num, ves_method = seg.segment_vesicles(
    img_ves, pixel_size, nuc_voxels=nuc_volume, method=method_vesicle
)
if nuc_num > 1:
    print(f'Multiple ({nuc_num}) nuclei were identfied',
          f'Only the biggest object was kept. Please check the nucleus segmentation.'
          )
if nuc_mask is None:
    raise RuntimeError(f'No nucleus detected')
if ves_num == 0:
    raise RuntimeError(f'No vesicles identified. '
                       f'Threshold method <{ves_method}> was used')
if ves_num == 1:
    print(f'Attention: only one vesicle was identified.')
# Visualise results
vis.add_image(img_dapi, "dapi", colormap="blue")
vis.add_image(img_ves, channel_name)
vis.add_labels(nuc_mask, "nucleus")
vis.add_labels(ves_mask, "vesicles")
# Measure
nuc_table = uts.measure_nucleus(nuc_mask, img_dapi, pixel_size, ves_num)
ves_table = uts.measure_vesicles(ves_mask, img_ves, pixel_size, ves_method)
dist_table = uts.measure_distances(nuc_mask, ves_mask, pixel_size)
angle_dict = uts.get_angle_and_distance(nuc_table, ves_table, pixel_size)
result_table = uts.merge_all_tables(nuc_table, ves_table, dist_table, angle_dict)
# Save Data
io.save_data(file_path, result_table, nuc_mask, ves_mask, img_dapi, img_ves, channel_name)
# Show table
result_table


## Batch process

In [ ]:
# Count images
total = 0
for file in os.listdir(dir_path):
    if (file.endswith('.tif') or file.endswith('.tiff') or file.endswith('.nd2')) and not '_mask_' in file and not '_nucCh' in file and not '_vesCh' in file:
        total += 1

count = 1
# Process images
for file in os.listdir(dir_path):
    if (file.endswith('.tif') or file.endswith('.tiff') or file.endswith('.nd2')) and not '_mask_' in file and not '_nucCh' in file and not '_vesCh' in file:
        print('Currently processing', file)
        path = os.path.join(dir_path, file)
        # Load image
        img, pixel_size = io.read_image(path)
        img_dapi = io.get_channel(img, dapi_ch)
        img_ves = io.get_channel(img, vesicle_ch)
        # Segment
        nuc_mask, nuc_num = seg.segment_nucleus(img_dapi, pixel_size)
        nuc_volume, nuc_volume_cal = uts.get_nucleus_volume(nuc_mask, pixel_size)
        ves_mask, ves_num, ves_method = seg.segment_vesicles(
            img_ves, pixel_size, nuc_voxels=nuc_volume, method=method_vesicle
        )
        if nuc_num > 1:
            print(
                f'Multiple ({nuc_num}) nuclei were identfied',
                f'Only the biggest object was kept. ',
                f'Please check the nucleus segmentation.'
                )
        if nuc_mask is None:
            raise RuntimeError(f'No nucleus detected')
        if ves_num == 0:
            raise RuntimeError(f'No vesicles identified. '
                            f'Threshold method <{ves_method}> was used')
        if ves_num == 1:
            print(f'Attention: only one vesicle was identified.')
        # Measure
        nuc_table = uts.measure_nucleus(nuc_mask, img_dapi, pixel_size, ves_num)
        ves_table = uts.measure_vesicles(ves_mask, img_ves, pixel_size, ves_method)
        dist_table = uts.measure_distances(nuc_mask, ves_mask, pixel_size)
        angle_dict = uts.get_angle_and_distance(nuc_table, ves_table, pixel_size)
        result_table = uts.merge_all_tables(nuc_table, ves_table, dist_table, angle_dict)

        # Save data
        io.save_data(
            path, result_table, nuc_mask, ves_mask, img_dapi, img_ves, channel_name
        )

        print('Finihsed image', count, 'out of', total, 'images.')
        count += 1

# Merge all tables
io.combine_csv_in_folder(dir_path, channel_name)